In [26]:
# Importing all the relevant packages.

import spacy #This statement should work if you have spaCy installed 
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report


from spacytextblob.spacytextblob import SpacyTextBlob
!python -m textblob.download_corpora

nlp = spacy.load('en_core_web_sm')


Finished.


[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\helde\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\helde\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\helde\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\helde\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package conll2000 to
[nltk_data]     C:\Users\helde\AppData\Roaming\nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\helde\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!

In [2]:
# Import the amazon reviews
data = pd.read_csv("amazon_product_reviews.csv", index_col=0)

reviews_data=data[['reviews.text']]

# cleaning the data:
clean_data=reviews_data.dropna()

In [3]:
clean_data=reviews_data.dropna(subset=['reviews.text'])

#Dropping the index column with the id. We will now only have 5000 numerical rows.
clean_data.reset_index(drop=True, inplace=True)
clean_data

,reviews.text
0,I thought it would be as big as small paper bu...
1,This kindle is light and easy to use especiall...
2,Didnt know how much i'd use a kindle so went f...
3,I am 100 happy with my purchase. I caught it o...
4,Solid entry level Kindle. Great for kids. Gift...
...,...
4995,This is a great tablet for the price. Amazon i...
4996,This tablet is the perfect size and so easy to...
4997,Purchased this for my son. Has room to upgrade...
4998,I had some thoughts about getting this for a 5...


Testing

In [4]:
nlp.add_pipe('spacytextblob')

# Attempting to extract a review as a text from the clean_data frame
first_review=clean_data['reviews.text'][0]
print(first_review)

doc = nlp(first_review)

print(doc._.blob.polarity )


I thought it would be as big as small paper but turn out to be just like my palm. I think it is too small to read on it... not very comfortable as regular Kindle. Would definitely recommend a paperwhite instead.
-0.10897435897435898


In [20]:
# Function to preprocess text using spaCy
def preprocess(text):
    doc = nlp(text)
    cleaned_tokens = [token.lemma_.lower() for token in doc if not token.is_stop and token.is_alpha]
    return " ".join(cleaned_tokens)


# Creating 3 empty lists to store the sentiment and reviews.
sent_score_list=[]      #Sentiment Score list, float
sent_type=[]            #Sentiment type, string -> Positive, Neutral , Negative
rev_list=[]             #Reviews list, string

# Looping through the reviews and placing them into a list.
for i in range(0,len(clean_data)):
    
    clean_data['reviews.text'][i]

    # Extracting the reviews and appending them to rev_list
    reviews=preprocess(clean_data['reviews.text'][i])
    rev_list.append(reviews)

    # Extracting the reviews and working out its polarity and appending it to the sent_score_list
    sentiment=nlp(preprocess(clean_data['reviews.text'][i]))
    sent_score_list.append(sentiment._.blob.polarity)

    # Adding the classification of the sentiment: positive, neutral and negative depending on the polarity score
    if sentiment._.blob.polarity==0:
        sent_type.append("Neutral")
    elif sentiment._.blob.polarity<0:
        sent_type.append("Negative")
    else:
        sent_type.append("Positive")




In [21]:
# converting the lists into numpy arrays
sent_score_arr = np.array(sent_score_list)
sent_type_arr = np.array(sent_type)
rev_arr = np.array(rev_list)

#creating 3 dataframes: Reviews , Sentiment_Score and Sentiment_Type
rev_df=pd.DataFrame(rev_arr)
sent_score_df=pd.DataFrame(sent_score_arr)
sent_type_df=pd.DataFrame(sent_type_arr)

# Concatenating the separate data frames into a single data frame
sent_analysis_df=pd.concat([rev_df,sent_score_df,sent_type_df],axis=1)
sent_analysis_df.columns=['Reviews','Sentiment_Score','Sentiment_Type']


,Reviews,Sentiment_Score,Sentiment_Type
0,think big small paper turn like palm think sma...,-0.016667,Negative
1,kindle light easy use especially beach,0.277778,Positive
2,not know use kindle go low end m happy little ...,0.115625,Positive
3,happy purchase catch sale good price normally ...,0.276786,Positive
4,solid entry level kindle great kid gifted kid ...,0.443333,Positive
...,...,...,...
4995,great tablet price amazon good job,0.750000,Positive
4996,tablet perfect size easy use read play game pu...,0.458333,Positive
4997,purchase son room upgrade memory allow book ga...,-0.400000,Negative
4998,thought get year old screen protector case fee...,0.025000,Positive


In [7]:
# # Function to preprocess text using spaCy
# def preprocess(text):
#     doc = nlp(text)
#     cleaned_tokens = [token.lemma_.lower() for token in doc if not token.is_stop and token.is_alpha]
#     return " ".join(cleaned_tokens)

In [8]:
# example_text="I thought it would be as big as small paper but turn out to be just like my palm. I think it is too small to read on it... not very comfortable as regular Kindle. Would definitely recommend a paperwhite instead."

# preprocess(example_text)

In [17]:
# for i in range(0,len(clean_data)):
    
#     preprocessed_sent=preprocess(clean_data['reviews.text'][i])
#     print(preprocessed_sent)

# preprocessed_sent=preprocess(clean_data['reviews.text'][0])
# type(preprocessed_sent)

str

In [22]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(sent_analysis_df['Reviews'], sent_analysis_df['Sentiment_Type'], test_size=0.2, random_state=42)

In [23]:
# Create a CountVectorizer with and without preprocessing
vectorizer_unprocessed = CountVectorizer()
X_train_unprocessed = vectorizer_unprocessed.fit_transform(X_train)
X_test_unprocessed = vectorizer_unprocessed.transform(X_test)

# vectorizer_processed = CountVectorizer(preprocessor=preprocess)
# X_train_processed = vectorizer_processed.fit_transform(X_train)
# X_test_processed = vectorizer_processed.transform(X_test)

In [24]:
# Train a Naive Bayes classifier on the unprocessed data
clf_unprocessed = MultinomialNB()
clf_unprocessed.fit(X_train_unprocessed, y_train)
y_pred_unprocessed = clf_unprocessed.predict(X_test_unprocessed)

print("Results without preprocessing:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_unprocessed)}")
print(classification_report(y_test, y_pred_unprocessed))

Results without preprocessing:
Accuracy: 0.89
              precision    recall  f1-score   support

    Negative       0.83      0.19      0.31        53
     Neutral       1.00      0.06      0.11        69
    Positive       0.89      1.00      0.94       878

    accuracy                           0.89      1000
   macro avg       0.91      0.41      0.45      1000
weighted avg       0.89      0.89      0.85      1000



In [25]:
# # Train a Naive Bayes classifier on the preprocessed data
# clf_processed = MultinomialNB()
# clf_processed.fit(X_train_processed, y_train)
# y_pred_processed = clf_processed.predict(X_test_processed)

# print("\nResults with preprocessing:")
# print(f"Accuracy: {accuracy_score(y_test, y_pred_processed)}")
# print(classification_report(y_test, y_pred_processed))


Results with preprocessing:
Accuracy: 0.889
              precision    recall  f1-score   support

    Negative       0.82      0.17      0.28        53
     Neutral       1.00      0.06      0.11        69
    Positive       0.89      1.00      0.94       878

    accuracy                           0.89      1000
   macro avg       0.90      0.41      0.44      1000
weighted avg       0.89      0.89      0.85      1000



In [14]:
sent_analysis_df["Sentiment_Type"][0]

'Negative'

In [28]:
X_test_processed[0]

<1x3360 sparse matrix of type '<class 'numpy.int64'>'
	with 6 stored elements in Compressed Sparse Row format>